In [1]:
import numpy as np
import networkx as nx

# from graphqec import RepetitionCode, RotatedSurfaceCode, CssCode, BivariateBicycleCode,commutation_test,row_sort
from graphqec import RepetitionCode, RotatedSurfaceCode, CssCode, BivariateBicycleCode, code_tools


In [2]:
bbc = BivariateBicycleCode(
    Lx = 6,
    Ly = 6,
    a1 = 3,
    a2 = 1,
    a3 = 2,
    b1 = 3,
    b2 = 1,
    b3 = 2,
)

A = bbc.A1+bbc.A2+bbc.A3
B = bbc.B1+bbc.B2+bbc.B3

Hx = [list(row) for row in np.hstack((bbc.A,bbc.B))]
Hz = [list(row) for row in np.hstack((bbc.B.T, bbc.A.T))]

cssbbc = CssCode(Hx,Hz)


In [5]:
logs = cssbbc._logic_check

In [6]:
len(logs[0]),len(logs[1])

(12, 12)

In [19]:
code_tools.commutation_test(logs[0],logs[1]),code_tools.commutation_test(cssbbc.Hz,logs[0]),code_tools.commutation_test(cssbbc.Hx,logs[1])

(False, True, True)

In [13]:
print(min([sum(row) for row in xlogs]))

10


In [6]:
kern = code_tools.compute_kernel(Hx)

In [7]:
len(kern)

42

In [8]:
ranking_rank = code_tools.binary_rank(Hz)
Hzw = [row for row in Hz]
for row in kern:
    if code_tools.binary_rank(Hzw+[row]) > ranking_rank:
        Hzw.append(row)
        ranking_rank = code_tools.binary_rank(Hzw)
        

In [9]:
len(Hzw)-len(Hz)

12

In [14]:
code_tools.commutation_test(Hx,Hzw[-12:])

True

In [42]:
len(Hx)

36

In [43]:
len(Hxn)

64

In [44]:
64-36

28

In [ ]:
for row in kern:
    print(len(row))

In [12]:
def gauss_elim(check_matrix):
    ncols = len(check_matrix[0])
    mtrans = code_tools.transpose_check(code_tools.row_sort(check_matrix))

    marks = []

    for col1 in range(ncols):
        if 1 in mtrans[col1]:
            mark = mtrans[col1].index(1)
            marks.append(mark)
            for col2 in range(ncols):
                if col2!= col1 and mtrans[col2][mark]:
                    mtrans[col2] = code_tools.add_rows(mtrans[col1],mtrans[col2])
    
    return code_tools.transpose_check(mtrans),marks

In [25]:
print(ms)

[0, 1, 4, 6, 8, 2, 12, 13, 16, 18, 20, 14, 24, 25, 28, 30, 32, 26, 5, 3, 17, 15, 29, 27, 10, 11, 7, 9, 19, 21]


In [26]:
len(ms)

30

In [27]:
len(Hx)

36

In [18]:
nzrows = 0
for row in Hxge:
    if not sum(row):
        nzrows+=1

In [11]:
Hxsteane = [[1,1,1,1,0,0,0],
      [0,1,1,0,1,1,0],
      [0,0,1,1,0,1,1],
      ]


In [12]:
cssst = CssCode(Hxsteane,Hxsteane)

In [13]:
stlgx = cssst._logic_check

In [14]:
for row in stlgx:
    print(row)

[[1, 0, 0, 1, 0, 0, 1]]
[[1, 0, 0, 1, 0, 0, 1]]


In [15]:
Hzsc = [[1,1,0,0,0,0,0,0,0],
      [0,1,1,0,1,1,0,0,0],
      [0,0,0,1,1,0,1,1,0],
      [0,0,0,0,0,0,0,1,1],
      ]

Hxsc = [[1,1,0,1,1,0,0,0,0],
      [0,0,1,0,0,1,0,0,0],
      [0,0,0,1,0,0,1,0,0],
      [0,0,0,0,1,1,0,1,1],
      ]

In [16]:
csssc =CssCode(Hxsc,Hzsc)

In [17]:
sclgx = csssc._logic_check

In [18]:
for row in sclgx:
    print(row)

[[1, 1, 0, 0, 0, 1, 0, 0, 0]]
[[1, 0, 0, 0, 1, 0, 0, 0, 1]]


In [7]:
def prep_matrix(check_matrix):
    cmsort = code_tools.row_sort(check_matrix)
    cmsextend = code_tools.row_extended_check(cmsort)
    hf = code_tools.transpose_check(cmsextend)
    return hf

In [37]:
def compute_kernel(check_matrix):
    nrows = len(check_matrix)
    ncols = len(check_matrix[0])

    marks = []

    hf = prep_matrix(check_matrix)

    for col1 in range(ncols):
        print(col1)
        mark = hf[col1].index(1)
        marks.append(mark)
        print(marks)
        for col2 in range(ncols):
            if col2 != col1:
                if hf[col2][mark]:
                    new_col = [(a+b)%2 for (a,b) in zip(hf[col1],hf[col2])]
                    hf[col2] = new_col
    
    return hf,marks
    

In [ ]:
marks = []
nrows = len(Hxsteane)
ncols = len(Hxsteane[0])

hsf = prep_matrix(Hxsteane)

for col1 in range(ncols):

    mark = hsf[col1].index(1)
    marks.append(mark)
    for col2 in range(ncols):

        if col2!= col1:
            if hsf[col2][mark]:
                new_row = code_tools.add_rows(hsf[col1],hsf[col2])
                hsf[col2] = new_row


In [20]:
def leading_ones(check_matrix: list[list[int]]) -> dict[int, list[int]]:
    r"""
    Function taking in a check matrix as a list of binary lists and
    outputs a dictionary mapping each index to the list of rows for
    which that index is the leading one in the list.

    :param check_matrix: list of binary lists

    """

    leaders = {}

    for row_index in range(len(check_matrix)):
        if check_matrix[row_index].index(1) in leaders:
            leaders[check_matrix[row_index].index(1)].append(row_index)
        else:
            leaders[check_matrix[row_index].index(1)] = [row_index]

    return leaders

In [21]:
def row_sort(check_matrix: list[list[int]]) -> list[list[int]]:
    r"""
    Function taking in a check matrix as a list of binary lists and outputs
    a sorted list of binary lists with the ordering determined by the
    compare_rows function.
    """

    num = len(check_matrix)
    if num < 2:
        return check_matrix
    else:
        first_half = row_sort(check_matrix[: int(num / 2)])
        second_half = row_sort(check_matrix[int(num / 2) :])
        sorted_full = []

        while len(first_half) > 0 and len(second_half) > 0:
            comp = compare_rows(first_half[0], second_half[0])
            if comp:
                sorted_full.append(second_half.pop(0))
            else:
                sorted_full.append(first_half.pop(0))

        return sorted_full + first_half + second_half


def compare_rows(row1: list[int], row2: list[int]) -> int:
    r"""
    A function that compares two rows based on the lowest unique non-zero element
    index.

    :param row1: binary list
    :param row2: binary list
    """

    assert len(row1) == len(row2)

    row1p2 = [(a + b) % 2 for (a, b) in zip(row1, row2)]

    if row1.index(1) == row1p2.index(1):
        return 0
    else:
        return 1


